# Relation between mean-CVaR and mean-variance optimization
This notebook illustrates the relationship between mean-CVaR and mean-variance optimization. It is the accompanying code for the Variance for Intuition, CVaR for Optimization article, available on https://ssrn.com/abstract=4034316.

You can watch a video walkthrough of the example here: https://youtu.be/gU4QD7PhZuk.

In the article, we argue that it is a desirable feature if mean-CVaR and mean-variance optimization give the same results when returns follow a multivariate normal distribution and only deviate if the left tails of the P&L simulations deviate significantly from a normal distribution. However, this is not the case for mean-CVaR optimization in general as the minimum risk portfolios can deviate quite significantly from each other depending on the expected returns. To illustrate this, we simulate 1,000,000 (!) scenarios from a multivariate normal distribution and show that demeaned mean-CVaR and mean-variance indeed converge to practically the same results. However, the non-demeaned CVaR converges to a significantly different solution that clearly prefers instruments with higher expected returns. Hence, the difference in frontier optimization results can be caused simply by the expected returns and not the difference in the left tail properties of the P&L simulation when one uses non-demeaned CVaR in mean-CVaR optimization. Unfortunately, demeaning CVaR for mean-CVaR optimization does not seem to be the current industry standard, and only few optimization technologies allow for the option to demean. In this package, returns are demeaned by default. The conditions for when mean-variance and mean-CVaR give the same results are given in Proposition 1 in https://doi.org/10.21314/JOR.2000.038.

In [1]:
import numpy as np
import pandas as pd
import fortitudo.tech as ft
from pypfopt.efficient_frontier import EfficientCVaR
from time import time

In [2]:
# Load parameters and simulate multivariate normal P&L
instrument_names, means, covariance_matrix = ft.load_parameters()
S = 1000000  # 1,000,000 scenarios is not recommended on Binder
R = np.random.default_rng(1).multivariate_normal(means, covariance_matrix, S)
means_sim = np.mean(R, axis=0)
print(f'Max absolute difference for means is {np.max(np.abs(means_sim - means))}.')
covariances_sim = np.cov(R, rowvar=False)
print(f'Max absolute difference for covariances is {np.max(np.abs(covariances_sim - covariance_matrix))}.')

Max absolute difference for means is 0.0004694984202286423.
Max absolute difference for covariances is 6.967627212783184e-05.


In [3]:
# Table 1 and 2
vols = np.sqrt(np.diag(covariance_matrix))
vols_inv = np.diag(vols**-1)
correlation = vols_inv @ covariance_matrix @ vols_inv
table1 = pd.DataFrame(
    100 * np.stack((means, vols)).T,
    index=instrument_names,
    columns=['Mean', 'Volatility'])
table2 = pd.DataFrame(100 * correlation, index=enumerate(instrument_names))
display(table1)
display(table2)

,Mean,Volatility
Gov & MBS,-0.70,3.2
Corp IG,-0.40,3.4
Corp HY,1.90,6.0
EM Debt,2.70,7.3
DM Equity,6.20,13.9
EM Equity,7.70,24.5
Private Equity,12.84,24.1
Infrastructure,5.70,10.2
Real Estate,4.23,7.8
Hedge Funds,4.69,6.9


,0,1,2,3,4,5,6,7,8,9
"(0, Gov & MBS)",100.0,60.0,0.0,30.0,-20.0,-10.0,-30.0,-10.0,-20.0,-20.0
"(1, Corp IG)",60.0,100.0,50.0,60.0,10.0,20.0,10.0,10.0,10.0,30.0
"(2, Corp HY)",0.0,50.0,100.0,60.0,60.0,70.0,60.0,30.0,30.0,70.0
"(3, EM Debt)",30.0,60.0,60.0,100.0,40.0,60.0,30.0,20.0,20.0,40.0
"(4, DM Equity)",-20.0,10.0,60.0,40.0,100.0,70.0,80.0,40.0,40.0,80.0
"(5, EM Equity)",-10.0,20.0,70.0,60.0,70.0,100.0,70.0,30.0,40.0,80.0
"(6, Private Equity)",-30.0,10.0,60.0,30.0,80.0,70.0,100.0,40.0,50.0,80.0
"(7, Infrastructure)",-10.0,10.0,30.0,20.0,40.0,30.0,40.0,100.0,40.0,40.0
"(8, Real Estate)",-20.0,10.0,30.0,20.0,40.0,40.0,50.0,40.0,100.0,50.0
"(9, Hedge Funds)",-20.0,30.0,70.0,40.0,80.0,80.0,80.0,40.0,50.0,100.0


In [4]:
# Specify long-only constraints
I = len(instrument_names)
G = -np.eye(I)
h = np.zeros(I)

# Specify optimization objects and compute efficient frontiers
opt_cvar = ft.MeanCVaR(R, G, h)
opt_cvar_mean = ft.MeanCVaR(R, G, h, options={'demean': False})
opt_var = ft.MeanVariance(means_sim, covariances_sim, G, h)
num_portfolios = 9
start_time = time()
frontier_var = opt_var.efficient_frontier(num_portfolios)
print(f'Variance efficient frontier with {I} instruments and {num_portfolios} portfolios'
    + f' computed in {np.round(time() - start_time, 2)} seconds.')
start_time = time()
frontier_cvar = opt_cvar.efficient_frontier(num_portfolios)
print(f'Demeaned CVaR efficient frontier with {S} scenarios, {I} instruments, and {num_portfolios} portfolios'
    + f' computed in {np.round(time() - start_time, 2)} seconds.')
start_time = time()
frontier_cvar_mean = opt_cvar_mean.efficient_frontier(num_portfolios)
print(f'Non-demeaned CVaR efficient frontier with {S} scenarios, {I} instruments, and {num_portfolios} portfolios' 
    + f' computed in {np.round(time() - start_time, 2)} seconds.')

Variance efficient frontier with 10 instruments and 9 portfolios computed in 0.03 seconds.
Demeaned CVaR efficient frontier with 1000000 scenarios, 10 instruments, and 9 portfolios computed in 17.97 seconds.
Non-demeaned CVaR efficient frontier with 1000000 scenarios, 10 instruments, and 9 portfolios computed in 17.93 seconds.


In [5]:
# Print efficient frontiers and compare (tables 3-5)
display(pd.DataFrame(np.round(100 * frontier_cvar, 2), index=instrument_names))
display(pd.DataFrame(np.round(100 * frontier_var, 2), index=instrument_names))
display(pd.DataFrame(np.round(100 * frontier_cvar_mean, 2), index=instrument_names))

,0,1,2,3,4,5,6,7,8
Gov & MBS,70.91,48.28,17.89,0.00,0.00,0.00,-0.00,-0.00,0.0
Corp IG,3.22,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.0
Corp HY,3.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Debt,0.00,0.18,8.12,1.77,-0.00,0.00,0.00,0.00,0.0
DM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Private Equity,0.00,-0.00,0.00,3.96,20.48,37.43,57.14,78.57,100.0
Infrastructure,1.82,9.46,16.68,28.27,40.40,52.54,42.86,21.43,-0.0
Real Estate,9.63,13.54,17.47,16.72,10.76,4.71,0.00,0.00,0.0
Hedge Funds,10.98,28.54,39.84,49.28,28.36,5.32,0.00,0.00,0.0


,0,1,2,3,4,5,6,7,8
Gov & MBS,70.92,48.19,17.86,0.00,0.00,0.00,0.00,0.00,0.0
Corp IG,3.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
Corp HY,3.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Debt,0.00,0.35,8.25,2.14,0.00,0.00,0.00,0.00,0.0
DM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
EM Equity,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0
Private Equity,0.00,0.00,0.00,4.03,20.46,37.40,57.15,78.58,100.0
Infrastructure,1.76,9.48,16.84,28.44,40.62,52.75,42.85,21.42,0.0
Real Estate,9.61,13.32,17.08,16.26,10.49,4.36,0.00,0.00,0.0
Hedge Funds,11.15,28.65,39.97,49.13,28.43,5.49,0.00,0.00,0.0


,0,1,2,3,4,5,6,7,8
Gov & MBS,62.14,35.11,6.53,-0.00,0.00,-0.00,-0.0,-0.0,0.0
Corp IG,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,-0.0
Corp HY,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
EM Debt,0.00,3.63,11.22,0.00,0.00,0.00,-0.0,0.0,0.0
DM Equity,-0.00,0.00,0.00,0.00,0.00,-0.00,0.0,0.0,0.0
EM Equity,0.00,-0.00,-0.00,0.00,0.00,0.00,0.0,0.0,0.0
Private Equity,0.00,-0.00,0.00,8.79,24.68,40.57,59.8,79.9,100.0
Infrastructure,5.46,12.62,19.37,32.06,43.39,54.89,40.2,20.1,0.0
Real Estate,11.51,15.11,18.91,15.01,9.21,3.50,0.0,0.0,0.0
Hedge Funds,20.89,33.53,43.97,44.15,22.72,1.04,0.0,0.0,0.0


# Computational efficiency comparison
Below, we also compare the results of our implementation based on https://doi.org/10.1007/s10287-005-0042-0 with results based on an implementation that uses the original formulation suggested in https://doi.org/10.21314/JOR.2000.038. This is simply to illustrate that we have implemented the efficient algorithm correctly and to give a sense of the computational efficiency gains. The interested reader can adjust the number of simulation S in this example and compare computational efficiency for smaller sample sizes. Note that pypfopt might produce user warnings regarding results being inaccurate.

In [6]:
# Comparison with pypfopt
target_return = np.mean(R @ frontier_var, axis=0)[0]
eff_cvar = EfficientCVaR(means_sim, R)
start_time = time()
port = eff_cvar.efficient_return(target_return)
print(f'pypfopt portfolio computed in {np.round(time() - start_time, 2)} seconds.')
start_time = time()
port2 = opt_cvar_mean.efficient_portfolio(target_return)
print(f'Non-demeaned P&L portfolio computed in {np.round(time() - start_time, 2)} seconds.')
start_time = time()
port3 = opt_cvar.efficient_portfolio(target_return)
print(f'Demeaned P&L portfolio computed in {np.round(time() - start_time, 2)} seconds.')
port4 = opt_var.efficient_portfolio(target_return)

/home/anton/miniconda3/envs/fortitudo.tech/lib/python3.9/site-packages/cvxpy/problems/problem.py:1296: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


pypfopt portfolio computed in 174.34 seconds.
Non-demeaned P&L portfolio computed in 7.38 seconds.
Demeaned P&L portfolio computed in 6.91 seconds.


In [7]:
# Compare results
port_values = np.array([value for value in port.values()])[:, np.newaxis]
pf_res = pd.DataFrame(
    np.round(100 * np.hstack((port_values, port2, port3, port4)), 2),
    index=instrument_names,
    columns=['Pypfopt', 'Non-demeaned', 'Demeaned', 'Variance'])
display(pf_res)

,Pypfopt,Non-demeaned,Demeaned,Variance
Gov & MBS,62.13,62.14,70.95,71.04
Corp IG,-0.00,0.00,3.12,2.90
Corp HY,-0.00,0.00,3.46,3.38
EM Debt,-0.00,0.00,0.00,0.00
DM Equity,-0.00,0.00,0.00,0.00
EM Equity,-0.00,-0.00,0.00,0.00
Private Equity,-0.00,0.00,0.00,0.00
Infrastructure,5.46,5.46,1.83,1.78
Real Estate,11.51,11.51,9.64,9.63
Hedge Funds,20.90,20.89,11.01,11.27


# License

In [8]:
# fortitudo.tech - Novel Investment Technologies.
# Copyright (C) 2021-2023 Fortitudo Technologies.

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.